# STIPS Advanced Usage Example

This notebook will demonstrate a number of ways of modifying STIPS observations. It assumes that you already have STIPS installed, and that you are able to create and run basic STIPS observations. If not, the STIPS Tutorial notebook is a recommended starting point. This notebook includes the following:

* [Configuring STIPS](#Configuring-STIPS)
* [STIPS Oversampling](#STIPS-Oversampling)
* [Adjusting PSF grid size](#The-STIPS-PSF-Grid)


In [ ]:
# General imports used for various purposes
import os
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table, Column

# import matplotlib and configure for notebooks
import matplotlib
from matplotlib import style
from matplotlib import pyplot as plt
%matplotlib notebook
matplotlib.rcParams['axes.grid'] = False
matplotlib.rcParams['image.origin'] = 'lower'

# Since the purpose of the notebook is to do a STIPS tutorial, import stips
import stips

stellar_parameters = {
                      'n_stars': 50000,
                      'age_low': 7.5e12, 
                      'age_high': 7.5e12,
                      'z_low': -2.0, 
                      'z_high': -2.0,
                      'imf': 'salpeter', 
                      'alpha': -2.35,
                      'binary_fraction': 0.1,
                      'clustered': True,
                      'distribution': 'invpow',
                      'radius': 100.0, 
                      'radius_units': 'pc',
                      'distance_low': 20.0, 
                      'distance_high': 20.0,
                      'offset_ra': 0.0, 
                      'offset_dec': 0.0
                     }

offset = {
          'offset_id': 1,
          'offset_centre': False,
          'offset_ra': 0.0,
          'offset_dec': 0.0,
          'offset_pa': 0.0
         }

observation_parameters = {
                          'instrument': 'WFI',
                          'filters': ['F129'],
                          'detectors': 1,
                          'distortion': False,
                          'observations_id': 1,
                          'exptime': 1,
                          'offsets': [offset]
                         }

print(stips.__env__report__)

# Utility function for creating a WFI WCS to turn catalogue (x,y) into (RA,DEC)
def make_wcs(ra, dec, pa):
    wfi_scale = 0.11 #arcsec/pixel
    w = WCS(naxis=2)
    w.wcs.ctype = ["RA---TAN", "DEC--TAN"]
    w.wcs.crpix = [2048, 2048]
    w.wcs.crval = [ra, dec]
    w.wcs.cdelt = [wfi_scale/3600., wfi_scale/3600.]
    pc_11 = np.cos(np.radians(pa))
    pc_12 = -np.sin(np.radians(pa))
    pc_21 = np.sin(np.radians(pa))
    pc_22 = np.cos(np.radians(pa))
    w.wcs.pc = [[pc_11, pc_12], [pc_21, pc_22]]
    return w

# Utility function for creating a sample point-source catalogue from
# image co-ordinates and fluxes
def create_point_source_catalogue(xs, ys, fluxes):
    wcs = make_wcs(0., 0., 0.)
    ra_dec = wcs.all_pix2world(xs, ys, 1)
    ras = ra_dec[0]
    decs = ra_dec[1]
    t = Table()
    t['id'] = Column(data=np.arange(len(ras), dtype=np.int32))
    t['ra'] = Column(data=ras)
    t['dec'] = Column(data=decs)
    t['flux'] = Column(data=fluxes)
    t['type'] = Column(data=['point']*len(ras))
    t['n'] = Column(data=[None]*len(ras))
    t['re'] = Column(data=[None]*len(ras))
    t['phi'] = Column(data=[None]*len(ras))
    t['ratio'] = Column(data=[None]*len(ras))
    t['notes'] = Column(data=[""]*len(ras))
    return t


## Configuring STIPS

STIPS can be configured in a variety of ways. These include:

* Creating or modifying the STIPS configuration file
* Creating configuration dictionaries
* Adding parameters to individual function calls

### Using the STIPS Configuration File

STIPS uses a yaml file for configuration. STIPS keeps a copy of the file in its data directory, and another copy (initially set to the STIPS default values) is located in the `stips_data` directory. When using a configuration file, STIPS will check its files in the following order:

1. Any file name supplied to the `stips.utilities.SelectParameter()` function, using the `config_file` keyword.
2. Any file named "stips_config.yaml" in the current directory
3. Any file pointed to by the `stips_config` environment variable (or, if that variable points to a directory, any
   file named "stips_config.yaml" in that directory).
4. Any file named "stips_config.yaml" in the `stips_data` directory
5. The "stips_config.yaml" file kept in the internal STIPS data directory

Note that, if a file is present in one of the above locations, but it doesn't include the configuration variable being checked, STIPS will continue through the list until it finds a file that does include the variable.

In the example below, one observation will be done with default settings, then a configuration file will be created setting the default background to 5 counts/s/pixel, then another observation will be taken.

In [ ]:
from stips.scene_module import SceneModule
from stips.observation_module import ObservationModule

if os.path.exists("stips_config.yaml"):
    os.remove("stips_config.yaml")

#config_text = "observation_default_background : 0.0\n"
#with open("stips_config.yaml", "w") as outf:
#    outf.write(config_text)

scm = SceneModule(out_prefix='example_scene', ra=0., dec=0., log_level="WARNING")
stellar_cat_file = scm.CreatePopulation(stellar_parameters)

obm = ObservationModule(observation_parameters, out_prefix="example_bkg_01", ra=0., dec=0., log_level="WARNING")
obm.nextObservation()
output_stellar_catalogues = obm.addCatalogue(stellar_cat_file)
psf_file = obm.addError()
first_fits_file, mosaic_file, params = obm.finalize(mosaic=False)
if os.path.exists("stips_config.yaml"):
    os.remove("stips_config.yaml")

config_text = "observation_default_background : 10.0\n"
with open("stips_config.yaml", "w") as outf:
    outf.write(config_text)
obm = ObservationModule(observation_parameters, out_prefix="example_bkg_02", ra=0., dec=0., log_level="WARNING")
obm.nextObservation()
output_stellar_catalogues = obm.addCatalogue(stellar_cat_file)
psf_file = obm.addError()
second_fits_file, mosaic_file, params = obm.finalize(mosaic=False)
os.remove("stips_config.yaml")

print(first_fits_file)
with fits.open(first_fits_file) as inf:
    for entry in inf[1].header['HISTORY']:
        if 'background' in entry.lower():
            print(entry)

print(second_fits_file)
with fits.open(second_fits_file) as inf:
    for entry in inf[1].header['HISTORY']:
        if 'background' in entry.lower():
            print(entry)


A list of configurable parameters and valid values can be found in the STIPS documentation.

## STIPS Oversampling

STIPS simulations can be internally oversampled at (theoretically) any level, although memory usage will increase as the square of the image oversample. Even though images are binned down to the actual detector resolution during the `stips.ObservationModule.addError()` function step, internal oversampling is important to obtain more accurate results. For example, image PSFs will have different shapes depending on where on the pixel the source falls, but this distinction is only visible in oversampled images. Below is an example of how to change image oversampling, along with the results of different levels of oversampling.

In [ ]:
# Create three point sources, all with the same flux (arbitrarily 1000 counts/s)
# - 1 at (200.0, 200.)
# - 1 at (400.5, 400.33)
# - 1 at (200.15, 399.7)
x_coords = [200., 400.5, 200.15]
y_coords = [200., 400.33, 399.7]
fluxes = [1000.]*len(x_coords)

# Use the utility function to make a table from these co-ordinates
sample_table = create_point_source_catalogue(x_coords, y_coords, fluxes)

# Because the output of interest is the three sample stars, turning
# off error residuals and background count rates will make that 
# feature the most obvious.
#
# In addition, creating a dictionary of parameters for a set of
# repeated observations will make it clearer which parameters are
# being changed.
observation_params = {
                        'ra': 0.,
                        'dec': 0.,
                        'log_level': 'WARNING',
                        'background': 5.,
                        'residual_poisson': False,
                        'residual_readnoise': False,
                        'residual_flat': False,
                        'residual_dark': False,
                        'residual_cosmic': False
                     }

obm = ObservationModule(observation_parameters, oversample=1, out_prefix='os1', seed=1, **observation_params)
obm.nextObservation()
output_tables = obm.addTable(sample_table, "internal")
psf_file = obm.addError()
result_oversample1, mosaic_file, params = obm.finalize(mosaic=False)
print("Finished Oversample 1")

obm = ObservationModule(observation_parameters, oversample=3, out_prefix='os3', seed=2, **observation_params)
obm.nextObservation()
output_tables = obm.addTable(sample_table, "internal")
psf_file = obm.addError()
result_oversample3, mosaic_file, params = obm.finalize(mosaic=False)
print("Finished Oversample 3")

We can plot the output from the above:

In [ ]:
with fits.open(result_oversample1) as result_file:
    res1 = result_file[1].data[100:500,100:500]
    res1 = np.where(res1>1.e-3, res1, 1.e-3)

with fits.open(result_oversample3) as result_file:
    res3 = result_file[1].data[100:500,100:500]
    res3 = np.where(res3>1.e-3, res3, 1.e-3)

res_sub = res3 - res1
res_sub = np.where(res_sub>1.e-3, res_sub, 1.e-3)

min_value = np.min([res1.min(), res3.min(), res_sub.min()])
max_value = np.max([res1.max(), res3.max(), res_sub.max()])
n = matplotlib.colors.LogNorm(vmin=min_value, vmax=max_value)

fig = plt.figure()
im1 = plt.imshow(res1, norm=n)
fig.suptitle("Oversample 1")
cb = fig.colorbar(im1)

fig = plt.figure()
im3 = plt.imshow(res3, norm=n)
fig.suptitle("Oversample 3")
cb = fig.colorbar(im3)

fig = plt.figure()
im_sub = plt.imshow(res_sub, norm=n)
fig.suptitle("Oversample 3 - 1")
cb = fig.colorbar(im_sub)

#fig1 = plt.figure()
#im = plt.matshow(result_1, norm=n)


In addition to the difference between the images, there are also differences between PSF shapes in the Oversample=3 image that are not present in the Oversample=1 image:

In [ ]:
im1_top_left = res1[:200,:200]
im1_top_right = res1[199:399, :200]
im1_diff = im1_top_left - im1_top_right

fig = plt.figure()
im1 = plt.imshow(im1_diff, norm=n)
fig.suptitle("Diff between 2 PSFs for Oversample 1")
cb = fig.colorbar(im1)

im3_top_left = res3[:200,:200]
im3_top_right = res3[198:398, :200]
im3_diff = im3_top_left - im3_top_right

fig = plt.figure()
im1 = plt.imshow(im3_diff, norm=n)
fig.suptitle("Diff between 2 PSFs for Oversample 3")
cb = fig.colorbar(im1)


## The STIPS PSF Grid

STIPS allows you to create grids of point-spread functions (PSFs) in order to incorporate the slight differences in the PSF over the detector. In order to effectively use a PSF grid, the PSF convolution must be broken down into more than a single step. This can be done either by increasing the "observation_detector_oversample" parameter (which will also increase output accuracy, although at the cost of increased processing time), decreasing the "psf_convolution_max_size" parameter (which will also reduce maximum memory consuption, although again at the cost of increased processing time). 

In [ ]:
# Create three point sources, all with the same flux (arbitrarily 1000 counts/s)
# - 1 at (200.0, 200.)
# - 1 at (400.5, 400.33)
# - 1 at (200.15, 399.7)
x_coords = [200., 2048., 3896.]
y_coords = [200., 2048., 3896.]
fluxes = [1000.]*len(x_coords)

# Use the utility function to make a table from these co-ordinates
sample_table = create_point_source_catalogue(x_coords, y_coords, fluxes)

# Because the output of interest is the three sample stars, turning
# off error residuals and background count rates will make that 
# feature the most obvious.
#
# In addition, creating a dictionary of parameters for a set of
# repeated observations will make it clearer which parameters are
# being changed.
observation_params = {
                        'ra': 0.,
                        'dec': 0.,
                        'log_level': 'WARNING',
                        'background': 5.,
                        'oversample': 1,
                        'psf_convolution_max_size': 2048,
                        'residual_poisson': False,
                        'residual_readnoise': False,
                        'residual_flat': False,
                        'residual_dark': False,
                        'residual_cosmic': False
                     }

obm = ObservationModule(observation_parameters, psf_grid_size=1, out_prefix='grid1', seed=1, **observation_params)
obm.nextObservation()
output_tables = obm.addTable(sample_table, "internal")
psf_file = obm.addError()
result_grid1, mosaic_file, params = obm.finalize(mosaic=False)
print("Finished Grid Size 1")

obm = ObservationModule(observation_parameters, psf_grid_size=2, out_prefix='grid2', seed=2, **observation_params)
obm.nextObservation()
output_tables = obm.addTable(sample_table, "internal")
psf_file = obm.addError()
result_grid2, mosaic_file, params = obm.finalize(mosaic=False)
print("Finished Grid Size 2")

obm = ObservationModule(observation_parameters, psf_grid_size=3, out_prefix='grid3', seed=2, **observation_params)
obm.nextObservation()
output_tables = obm.addTable(sample_table, "internal")
psf_file = obm.addError()
result_grid3, mosaic_file, params = obm.finalize(mosaic=False)
print("Finished Grid Size 3")

For example, looking at the single-point grid (same PSF everywhere), subtracting two PSFs from one another yields nothing:

In [ ]:
with fits.open(result_grid1) as result_file:
    res1 = result_file[1].data

n = matplotlib.colors.LogNorm(vmin=max(res1.min(), 1.e-10), vmax=res1.max())

r1 = res1[126:276, 126:276]
fig = plt.figure()
im1 = plt.imshow(r1, norm=n)
cb = fig.colorbar(im1)

r2 = res1[1973:2123, 1974:2124]
fig = plt.figure()
im1 = plt.imshow(r2, norm=n)
cb = fig.colorbar(im1)

sub1 = np.abs(r1 - r2)
fig = plt.figure()
im1 = plt.imshow(sub1)
cb = fig.colorbar(im1)


However, with the 3x3 grid this is not the case:

In [ ]:
with fits.open(result_grid3) as result_file:
    res3 = result_file[1].data

n = matplotlib.colors.LogNorm(vmin=max(res3.min(), 1.e-10), vmax=res3.max())

r1 = res3[126:276, 126:276]
fig = plt.figure()
im1 = plt.imshow(r1, norm=n)
cb = fig.colorbar(im1)

r2 = res3[1973:2123, 1974:2124]
fig = plt.figure()
im1 = plt.imshow(r2, norm=n)
cb = fig.colorbar(im1)

sub1 = np.abs(r1 - r2)
fig = plt.figure()
im1 = plt.imshow(sub1)
cb = fig.colorbar(im1)